In [ ]:
"""
*서로다른 알고리즘으로 형성 : 보팅 , 하나의 알고리즘으로 형성 : 배깅
*부스팅(정형 데이터에 적합) : 이전 분류기의 학습 결과 바탕으로 다음 분류기의 학습 데이터의 샘플 가중치 조정해 학습하는 방법
-XGBoost가 GradientBoost보다 오버피팅 방지에 적합하고, 속도가 훨씬 빠르다
-XGBoost는 GridSearchCv할시에는 파라미터가 많다보니 오래걸린다.
-XGBoost는 균형 잡힌 트리 이용(양쪽 트리에 대한 깊이 균등하게 = Level-wise tree growth) -> 오버피팅 줄이기 위해 -> 속도는 느려짐
-LIghtGradiendBoost는 GridSearchCv할시에 속도는 빠른데 자료가 많으면 과대적합에 걸린다 -> 10000개 데이터 미만 있어야 오키
-LightGBM은 Loss 가 가장 큰 노드를 선택하여 Subtree를 구성한다 = Leaf-wise tree growth ->속도 빠름  

*스태킹
각각의 모델에다가 test데이터를 집어넣어 예측을 하는데 이러한 데이터를 다시 training 데이터로 사용
    -> Svm      -> 예측값
data-> RF       -> 예측값 => training data로 사용
    -> Light GBM-> 예측값
Cv기반 stacking = 각각의 모델들이 교차검증 기반으로 학습데이터 생성


*데이터 불균형 : 클래스가 어느 한쪽으로만 일방적으로 존재
해결방법
1) 오버 샘플링 : 클래스가 적은 쪽의 데이터를 랜덤 복원 샘플링하여 복사 붙여넣기를 반복하여 두 클래스의 비율을 비슷하게함

2) 언더 샘플링 : 클래스가 많은 쪽의 데이터를 랜덤 샘플링하여 삭제하기를 반복하여 두 클래스의 비율을 비슷하게 함

3) 오버 & 언더 샘플링 
ex) Y:1000건 vs N:10건 => 1010/2 = 505, Y는 505건이 될때까지 언더 샘플링, N은 505건이 될때까지 오버 샘플링한다.

4) SMOTE 알고리즘
기존 데이터를 적절하게 혼합하여 새로운 데이터를 생성하는 방법

ROC커브 이용해 모델 정확도 확인 
ROC커브 : 민감도(SE)와 1-특이도(SP)로 그린 그림
 => 1-특이도(SP)=1- TN/TN+FP = FP/TN+FP = 실제 음성인 것과 양성으로 잘못 예측한 것의 비율 = 위양성율 (FPR)
AUC커브는 Roc커브의 아래쪽 면적이고 이 값은 클수록 좋은 모델이다

민감도(Sensitivity=SE) = TP/TP+FN = 맞춘 양성/실제 양성 = FN이 작을수록 정확성이 증가, 민감도는 클수록 좋다
특이도(Specificity=SP) = TN/TN+FP = 맞춘 음성/실제 음성 = FP이 작을수록 좋고, 특이도가 클수록 좋은 모델

*Linear Regression(선형회귀) = 데이터들의 분포가 직선형태
- cost function값이 작다 = 직선과 점사이의 거리가 가깝다
- cost가 작을수록 좋은 모델이라 할수 있음
- h(x) - y = 예측값 -실제값 = ERROR
- 우리가 찾는값 = cost를 작게하는 h(x)값 -> W,b를 찾는다
- W랑 b를 조금씩 바꿔가면서 cost값을 줄인다
- cost가 최솟값을 갖았다 판단되면 중단

*sigmoid 함수 : 모든 값을 0~1사이로 나타낸다 -> 0.5보다 크면 1, 작으면 0, 기존의 cost 함수 사용 못해서 log함수 그래프로 사용 

*Softmax Cross-Entropy : 다중분류할경우 사용
- softmax function : 각각의 분류기로부터 값을 전달받아 확률로 변환해주는 함수 -> 가장 높은 확률을 갖는 값을 정답으로 예측
                     이후 가장 높은 값의 확률을 w,b를 조정해 1로 나머지는 0으로 만듬

linear regression -> sigmoid이용한 binaryclassification(logits) ->  softmax

*Association Rule Mining(연관분석)

Itemset ; 항목집합 -> 하나이상의 아이템들의 집합 
Support count : 항목집합의 발생 빈도수 => 항목집합의 아이템이 모두 다있는 거래가 몇건인지 
Support : 항목집합에 대한 카운트(Support count)를 전체 거래에 대해 비중으로 나타낸것
Frequent itemset: 빈발항목 집합 => 최소 지지 인계값(mnsup)이상에 해당하는 항목집합
                  ex) mnsup를 2로 설정하면 support count가 2이상일경우 빈발항목 집합이라 한다

Association Rule(연관규칙) : x라고 하는 항목집합에 대한 구매가 발생됐을때 y라는 항목에 대해 구매가 이뤄진다
Support(지지도) : X,Y 라는 두개의 아이템 집합을 모두 포함하는 거래가 전체중에 얼마나 있나 비율 
confidence(신뢰도) : 조건부 확률 -> x상품이 구매됐을때 y가 구매될 확률이 얼마나 되나? -> X라고 하는 항목집합의 거래 수가 분모로 온다
                   -> x,y둘다 포함하는 집합 수/ x만 포함하는 집합 수
lift(향상도) : 향상도 1에 가까우면 관게없음  ex)lift(막걸리->파전) = (막걸리 -> 파전)신뢰도/(파전)지지도 or 
                                                  막걸리, 파전 모두 구매한 거래수/ (막걸리만 거래한 거래수 * 피전만 가레힌 거래수)
leverage(삼겹살 -> 상추) = p(삼겹살, 상추 교집합) - p(삼겹살)*p(상추) 
-> leverage가 0에 가까우면 두 상품은 독립, 0보다 크다면(향상도가 1보다 큰경우) 두 상품은 연관성 있다

Conviction : Convict(삼겹살->상추) : p(상추 구매 안하는경우)/p(삽겹살 구매한경우중 상추는 구매안할경우)
->값이 1이면 서로 독립이다(관련없다), 1보다 크다면 삼겹살은 상추 구매여부 예측하는데 유용하다.
  1보다 작으면 유용하지 않다


연관분석 필요성 : 대용량 데이터 베이스에서 기존에는 발견할수 없는 아이템간의 관계를 발견할수 있다.

빈발 항목 아이템 집합 생성 : support >= minsup

빈발 항목 아이템으로부터 룰이 나온다 -> but 각각의 아이템셋으로부터 나올수 있는 규칙이 너무 많다.
                   
Apriori 알고리즘 => 설정한 지지도를 만족하면 빈발 항목 집합이고, 만족 못하면 비빈발 항목 집합이다
                 => 지지도에 부합이 안되는 아이템셋아 있다면 해당 아이템셋이 포함되어있는 모든 아이템셋을 비빈발 항목집합으로 보고 제외시킴
                 
"""

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
data=pd.read_csv('titanic/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data['Embarked'].fillna('S', inplace = True)
data['Fare'].fillna(0, inplace=True)
data['Fare'] = data['Fare'].map(lambda x : np.log(x) if x > 0 else 0)

In [5]:
data['Initial'] = data['Name'].str.extract('([A-Za-z]+)\.')
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Other'],inplace=True)
mapping = {
    "Mr":0,
    "Miss":1,
    "Mrs" : 1,
    "Master":2,
    "Other":3
}

data['Initial'] = data['Initial'].map(mapping)

In [6]:
mapping_sex = {
    'male' : 0,
    'female': 1
}

mapping_em = {
    'S' :0,
    'C' :1,
    'Q' :2
}


data['Sex'] = data['Sex'].map(mapping_sex)
data['Embarked'] = data['Embarked'].map(mapping_em)


data.drop(['PassengerId', "Ticket", "Cabin", "Name"], axis = 1, inplace = True)

In [7]:
data.groupby('Initial')['Age'].mean()

Initial
0    32.739609
1    27.834615
2     4.574167
3    45.888889
Name: Age, dtype: float64

In [8]:
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 0), 'Age' ] = 32
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 1), 'Age' ] = 28
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 2), 'Age' ] = 5
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 3), 'Age' ] = 45

In [9]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial
0,0,3,0,22.0,1,0,1.981001,0,0
1,1,1,1,38.0,1,0,4.266662,1,1
2,1,3,1,26.0,0,0,2.070022,0,1
3,1,1,1,35.0,1,0,3.972177,0,1
4,0,3,0,35.0,0,0,2.085672,0,0


In [10]:
y = data['Survived']
X = data.drop('Survived', axis = 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [12]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [13]:
pred = rf.predict(X_test)
print("정확도 :{0:.3f}".format(accuracy_score(y_test, pred)))

정확도 :0.832


In [14]:
gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [15]:
#GridSearch를 위한 파라미터
gb_param_grid = {
    'n_estimators' : [100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [3, 5, 7, 10],
    'min_samples_split' : [2, 3, 5, 10]
}

In [16]:
gb_grid = GridSearchCV(gb, param_grid = gb_param_grid, scoring="accuracy", n_jobs= -1, verbose = 1)
gb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


KeyboardInterrupt: 

In [ ]:
gb_grid.best_score_

In [ ]:
#파라미터 이렇게 설정하면 정확도 높다
gb_grid.best_params_

In [ ]:
#가장 좋은 파라미터들로 모델 생성
#타이타닉의 테스트 데이터를 입력
#결과를 제출

In [ ]:
test=pd.read_csv('titanic/test.csv')
test

In [ ]:
test.isnull().sum()

In [44]:
data['Embarked'].fillna('S', inplace = True)
test['Fare'].fillna(0, inplace=True)
test['Fare'] = data['Fare'].map(lambda x : np.log(x) if x > 0 else 0)

In [45]:
test['Initial'] = test['Name'].str.extract('([A-Za-z]+)\.')
test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Other'],inplace=True)
mapping = {
    "Mr":0,
    "Miss":1,
    "Mrs" : 1,
    "Master":2,
    "Other":3
}

test['Initial'] = test['Initial'].map(mapping)

In [46]:
mapping_sex = {
    'male' : 0,
    'female': 1
}

mapping_em = {
    'S' :0,
    'C' :1,
    'Q' :2
}


test['Sex'] = test['Sex'].map(mapping_sex)
test['Embarked'] = test['Embarked'].map(mapping_em)


test.drop(['PassengerId', "Ticket", "Cabin", "Name"], axis = 1, inplace = True)

In [47]:
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 0), 'Age' ] = 32
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 1), 'Age' ] = 28
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 2), 'Age' ] = 5
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 3), 'Age' ] = 45

In [48]:
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial
0,3,0,34.5,0,0,0.683603,2,0
1,3,1,47.0,1,0,1.450832,0,1
2,2,0,62.0,0,0,0.727559,2,0
3,3,0,27.0,0,0,1.379314,0,0
4,3,1,22.0,1,1,0.735091,0,1
...,...,...,...,...,...,...,...,...
413,3,0,32.0,0,0,0.000000,0,0
414,1,1,39.0,0,0,0.727559,1,3
415,3,0,38.5,0,0,0.735091,0,0
416,3,0,32.0,0,0,1.247389,0,0


In [52]:
gb2=GradientBoostingClassifier(max_depth= 6,
 min_samples_leaf= 10,
 min_samples_split= 2,
 n_estimators= 100)

In [53]:
gb2.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=6, min_samples_leaf=10)

In [57]:
pred=gb2.predict(test)
sub= pd.read_csv('titanic/test.csv')[['PassengerId']]
pred=pd.Series(pred)
sub['Survived']=pred
sub.to_csv('sub_titanic',index=False)

In [ ]:
###ROC curve #####

In [17]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [19]:
card_df = pd.read_csv('creditcard.csv')
card_df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [20]:
#train과 test 자료가 나눠져있지 않는 경우 train_test_split import 해서 나눠줌
#나눠줄떄 random_state 사용해서 지정해줘야함
#train이나 test 둘중 하나 size 지정
from sklearn.model_selection import train_test_split

In [21]:
x_features=card_df.iloc[:,:-1]  # class배고 추출 = 독립변수 :x
y_target=card_df.iloc[:,-1]   # class 만 추출 = 종속변수 :y

In [22]:
#train 데이터에 대해 x,y test데이터에 대해 x,y 총 4개로 나눠짐
xtrain,xtest,ytrain,ytest=train_test_split(x_features, y_target, test_size=0.3, random_state=20231023, stratify=y_target)  
#stratify 층화추출 = 원데이터의 클래스 비율 유지하면서 나눠준다(7:3)

In [23]:
ytrain.value_counts()

0    199020
1       344
Name: Class, dtype: int64

In [24]:
ytest.value_counts()

0    85295
1      148
Name: Class, dtype: int64

In [25]:
#copy함수 사용함 , copy 데이터 정제
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    df_copy.drop('Time', axis=1, inplace=True)
    return df_copy

In [26]:
df_copy = get_preprocessed_df(card_df)

In [27]:
df_copy

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [28]:
def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)                                                

In [30]:
X_train.head()
X_test.head()


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
170978,0.101337,0.665808,-0.514168,-0.675752,1.087999,-0.681790,0.877323,-0.096356,0.265313,-1.297842,...,-0.177718,0.114197,0.500593,-0.066669,0.558720,-0.518068,0.367070,0.079707,0.131678,12.50
96185,-1.678212,1.244794,0.947874,-2.101099,-0.811015,-1.037444,0.144664,0.465090,-1.137320,0.847260,...,-0.246449,-0.383548,-0.581462,0.033139,0.483823,-0.039289,0.078483,0.486383,0.291222,10.00
30469,-1.407998,-0.094725,1.218361,-1.884626,0.639017,-1.127801,0.537467,-0.295973,-1.670308,0.022252,...,-0.126990,0.295294,0.836930,-0.367286,0.157457,0.840789,-0.160280,-0.210861,0.103780,29.95
266925,2.025163,-1.001783,-0.855068,-0.506527,-0.793876,-0.197866,-0.779789,0.032557,-0.357041,1.015046,...,-0.519001,-0.298900,-0.427423,0.298094,0.747735,-0.426508,0.500501,-0.051892,-0.048338,50.27
14185,0.477071,-1.295023,0.586617,1.599474,-1.215659,0.114858,-0.178181,0.090062,0.944582,-0.207109,...,0.543494,0.299761,0.114989,-0.461395,-0.122341,0.381628,-0.245921,-0.013775,0.092648,380.00


In [31]:
print(y_train.value_counts()/y_train.shape[0] * 100)

0    99.827451
1     0.172549
Name: Class, dtype: float64


In [32]:
print(y_test.value_counts()/y_test.shape[0] * 100)

0    99.826785
1     0.173215
Name: Class, dtype: float64


In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
#객체 생성
lr_clf=LogisticRegression()

In [35]:
lr_clf.fit(X_train,y_train)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [36]:
lr_clf.predict(X_test)
pd.Series(lr_clf.predict(X_test)).value_counts()

0    85341
1      102
dtype: int64

In [37]:
lr_clf.predict_proba(X_test)    #proba = 확률로 나온것 -> 같은 행 합이 1이된다.

array([[9.98651690e-01, 1.34831002e-03],
       [9.99876546e-01, 1.23454097e-04],
       [9.99808217e-01, 1.91783045e-04],
       ...,
       [9.99753313e-01, 2.46686772e-04],
       [9.99253336e-01, 7.46664288e-04],
       [9.99865249e-01, 1.34751489e-04]])

In [38]:
lr_clf.predict_proba(X_test)[:,1]

array([0.00134831, 0.00012345, 0.00019178, ..., 0.00024669, 0.00074666,
       0.00013475])

In [40]:
pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b3/f8/ee33e36194eb03a76eccf3adac3fba51f0e56fbd20609bb531659d48d3cb/lightgbm-4.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------  1.3/1.3 MB 42.0 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 42.0 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 16.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [41]:
from lightgbm import LGBMClassifier

In [43]:
lgbm_clf=LGBMClassifier(n_estimators=1000, num_leaves=64, boost_from_average=False)  #트리 갯수 1000개, 리프노드 갯수 64개
                                                                            #boost_from_average 는 데이터가 불균형 분포되어있을때 False
                                                                            #                 -> 균형있을떄는 True

In [44]:
#모델 만듬
lgbm_clf.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 344, number of negative: 199020
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 199364, number of used features: 29
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)

In [46]:
#test 데이터에 대한 예측 결과
pred=lgbm_clf.predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [48]:
lgbm_clf.predict_proba(X_test)  #확률로 출력

array([[9.99999997e-01, 3.49830240e-09],
       [1.00000000e+00, 1.08517297e-10],
       [1.00000000e+00, 2.23232317e-10],
       ...,
       [9.99999996e-01, 3.66393546e-09],
       [9.99999999e-01, 6.63996306e-10],
       [1.00000000e+00, 1.08138237e-10]])

In [52]:
import numpy as np

In [58]:
#이상치 제거 
X_train.head()
X_train.describe()
q25=np.percentile(X_train['V1'].values,25)  # ->1사분위수
q75=np.percentile(X_train['V1'].values,75)  # ->3사분위수
iqr=q75-q25
iqr15=iqr*1.5

#데이터 표준화




In [62]:
lowest_val=q25-iqr15  # 하한 바운더리
highest_val=q75+iqr15  # 상한 바운더리

In [65]:
#이상치 
X_train['V1'][(X_train['V1']<lowest_val) | (X_train['V1']>highest_val)].index  #인덱스만 뽑아서 전처리도 가능

Int64Index([211605, 151631,  76190, 207772, 220170, 109519, 166031, 283347,
             18527, 259701,
            ...
            222507,  72745,  20946,  77564, 169338, 125442, 264145, 216442,
             20260,  19760],
           dtype='int64', length=4928)

In [ ]:
#SMOTE 오버 샘플링  -> 데이터 불균형으로 인해 데이터 오버 샘플링 실행

In [67]:
pip install -U imbalanced-learn

  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/a3/9e/fbe60a768502af54563dcb59ca7856f5a8833b3ad5ada658922e1ab09b7f/imbalanced_learn-0.11.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   ---------------------------------------  235.5/235.6 kB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 235.6/235.6 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1
Note: you may need to restart the kernel to use updated packages.


In [68]:
from imblearn.over_sampling import SMOTE

In [69]:
smote=SMOTE(random_state=0)

In [70]:
X_train_over, y_train_over=smote.fit_resample(X_train,y_train)  #oversampling 실행

In [71]:
X_train.shape

(199364, 29)

In [72]:
X_train_over.shape

(398040, 29)

In [75]:
pd.Series(y_train_over).value_counts()  #오버트렝이닝을 통해 1의 데이터가 100건정도에서 190000건으로 증가

0    199020
1    199020
Name: Class, dtype: int64

In [76]:
lr_clf=LogisticRegression()

In [77]:
lr_clf.fit(X_train_over,y_train_over)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [78]:
lr_clf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [79]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [85]:
cancer_data=load_breast_cancer()
cancer_data
X_data=cancer_data.data   # 데이터만 추출
y_label=cancer_data.target

In [89]:
X_train,X_test,y_train,y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=20231024)

In [92]:
#스태킹(stack)
#기본모델 (knn, randomforest, dt, adaboost)
#스태킹을 위해 기본 모델 4개 생성
knn_clf=KNeighborsClassifier(n_neighbors=5)
rf_clf=RandomForestClassifier(n_estimators=100, random_state=42)
dt_clf=DecisionTreeClassifier()
ada_clf=AdaBoostClassifier(n_estimators=100)

In [93]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [94]:
knn_pred=knn_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
dt_pred=dt_clf.predict(X_test)
ada_pred=ada_clf.predict(X_test)

In [96]:
print(accuracy_score(y_test,knn_pred))
print(accuracy_score(y_test,rf_pred))
print(accuracy_score(y_test,dt_pred))
print(accuracy_score(y_test,ada_pred))

0.9298245614035088
0.9473684210526315
0.9473684210526315
0.9736842105263158


In [103]:
#알고리즘별로 예측값을 모아서 다시 train data로 구성해야한다

pred=np.array([knn_pred,rf_pred,dt_pred,ada_pred]) #(114)=> (4,114)
pred

array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        1, 1, 1, 1],
       [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,

In [104]:
pred=np.transpose(pred)   # => 
pred  #(114,4)

array([[1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1,

In [105]:
#최종 스태킹 모델 : 로지스틱 회귀
lr_final=LogisticRegression()

In [106]:
lr_final.fit(pred, y_test)

LogisticRegression()

In [107]:
final= lr_final.predict(pred)
final

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1])

In [108]:
accuracy_score(final,y_test)

0.9736842105263158

In [ ]:
#연관규칙

In [110]:
pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/73/da/d5d77a9a7a135c948dbf8d3b873655b105a152d69e590150c83d23c3d070/mlxtend-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------- ----------------------------- 0.4/1.4 MB 12.2 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 10.2 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 10.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [111]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [160]:
dataset=[['사과','치즈','생수'],
['생수','호두','치즈','고등어'],
['수박','사과','생수'],
['생수','호두','치즈','옥수수']]

In [161]:
te=TransactionEncoder()

In [162]:
te_ary=te.fit(dataset).transform(dataset)

In [163]:
te.columns_

['고등어', '사과', '생수', '수박', '옥수수', '치즈', '호두']

In [164]:
te_ary
df=pd.DataFrame(te_ary, columns = te.columns_)
df    # 첫번째부터 네번째 거래까지 True에 해당하는 값들이 구매한 내역

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [165]:
apriori(df,min_support=0.5, use_colnames=True)                #min_support = 최소 지지도 - > 설정 값에 따라 나오는 아이템이 달라짐 
# 지지도가 0.5 이상인 아이템셋들이 나옴 -> 빈발 항목 집합

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(사과, 생수)"
5,0.75,"(치즈, 생수)"
6,0.50,"(호두, 생수)"
7,0.50,"(호두, 치즈)"
8,0.50,"(호두, 치즈, 생수)"


In [166]:
freq_itemsets=apriori(df,min_support=0.5, use_colnames=True)
freq_itemsets

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(사과, 생수)"
5,0.75,"(치즈, 생수)"
6,0.50,"(호두, 생수)"
7,0.50,"(호두, 치즈)"
8,0.50,"(호두, 치즈, 생수)"


In [167]:
association_rules(freq_itemsets) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(사과),(생수),0.50,1.00,0.50,1.0,1.000000,0.000,inf,0.0
1,(치즈),(생수),0.75,1.00,0.75,1.0,1.000000,0.000,inf,0.0
2,(호두),(생수),0.50,1.00,0.50,1.0,1.000000,0.000,inf,0.0
3,(호두),(치즈),0.50,0.75,0.50,1.0,1.333333,0.125,inf,0.5
4,"(호두, 치즈)",(생수),0.50,1.00,0.50,1.0,1.000000,0.000,inf,0.0
5,"(호두, 생수)",(치즈),0.50,0.75,0.50,1.0,1.333333,0.125,inf,0.5
6,(호두),"(치즈, 생수)",0.50,0.75,0.50,1.0,1.333333,0.125,inf,0.5


In [168]:
res=association_rules(freq_itemsets,metric='lift') 
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf,0.0
1,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0,0.0
2,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf,0.0
3,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0,0.0
4,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf,0.0
5,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0,0.0
6,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf,0.5
7,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5,1.0
8,"(호두, 치즈)",(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf,0.0
9,"(호두, 생수)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf,0.5


In [ ]:
"""
연습문제

마트의 데이터 분석가 
 - 번들, 유통, 대체 등
"""

In [170]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

In [171]:
te=TransactionEncoder()

In [172]:
te_ary=te.fit(dataset).transform(dataset)

In [173]:
te.columns_

['Apple',
 'Corn',
 'Eggs',
 'Ice cream',
 'Milk',
 'Nutmeg',
 'Onion',
 'Unicorn',
 'Yogurt']

In [174]:
df=pd.DataFrame(te_ary,columns=te.columns_)
df

,Apple,Corn,Eggs,Ice cream,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,True,False,True,True,True,False,True
1,False,False,True,False,False,True,True,False,True
2,True,False,True,False,True,False,False,False,False
3,False,True,False,False,True,False,False,True,True
4,False,True,True,True,False,False,True,False,False


In [181]:
freq_itemsets=apriori(df,min_support=0.4,use_colnames=True)
freq_itemsets

,support,itemsets
0,0.4,(Corn)
1,0.8,(Eggs)
2,0.6,(Milk)
3,0.4,(Nutmeg)
4,0.6,(Onion)
5,0.6,(Yogurt)
6,0.4,"(Eggs, Milk)"
7,0.4,"(Eggs, Nutmeg)"
8,0.6,"(Eggs, Onion)"
9,0.4,"(Yogurt, Eggs)"


In [182]:
res=association_rules(freq_itemsets,metric='lift')
res[res['lift']>1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(Eggs),(Nutmeg),0.8,0.4,0.4,0.500000,1.250000,0.08,1.2,1.000000
3,(Nutmeg),(Eggs),0.4,0.8,0.4,1.000000,1.250000,0.08,inf,0.333333
4,(Eggs),(Onion),0.8,0.6,0.6,0.750000,1.250000,0.12,1.6,1.000000
5,(Onion),(Eggs),0.6,0.8,0.6,1.000000,1.250000,0.12,inf,0.500000
8,(Yogurt),(Milk),0.6,0.6,0.4,0.666667,1.111111,0.04,1.2,0.250000
9,(Milk),(Yogurt),0.6,0.6,0.4,0.666667,1.111111,0.04,1.2,0.250000
10,(Onion),(Nutmeg),0.6,0.4,0.4,0.666667,1.666667,0.16,1.8,1.000000
11,(Nutmeg),(Onion),0.4,0.6,0.4,1.000000,1.666667,0.16,inf,0.666667
12,(Yogurt),(Nutmeg),0.6,0.4,0.4,0.666667,1.666667,0.16,1.8,1.000000
13,(Nutmeg),(Yogurt),0.4,0.6,0.4,1.000000,1.666667,0.16,inf,0.666667


In [ ]:
달걀 지지도 -> 0.8 -> 혼자서도 잘 팔림 -> 많이 진열할수록 좋다
넛맥 지지도 -> 0.4 -> 혼자서는 잘 안팔림 
달걀, 요거트, 양파, 넛맥  -> 번들로 팔면 좋음 -> 아침식사
달걀, 요거트, 양파, 넛맥은 주로 같이 팔리고 번들로 팔면 좋음 